[View in Colaboratory](https://colab.research.google.com/github/hjjimmykim/prospectiveporpoise/blob/master/Untitled0.ipynb)

Base iterated prisoner's dilemma game

# Libraries

In [0]:
# Standard
import numpy as np
import matplotlib.pyplot as plt
import copy

import tensorflow as tf

# cuda
use_cuda = 1

# Parameters

In [0]:
#Reward structure
T = 5 # Temptation
R = 3 # Reward
P = 1 # Penalty
S = 0 # Sucker

# Reward matrix
RM = np.zeros([2,2,2])
RM[0][0][0] = R
RM[0][0][1] = R
RM[0][1][0] = S
RM[0][1][1] = T
RM[1][0][0] = T
RM[1][0][1] = S
RM[1][1][0] = P
RM[1][1][1] = P

# RL
gamma = 0.95
alpha = 0.1

# Tensorflow stuff
n_eps = 100       # Number of episodes
n_turns = 10      # Number of turns per episode
n_hidden  = 10
layers = 1
epsilon = 0.1       # Exploration parameter (epsilon-greedy)
epslion_dr = 0.9999
input_dim = 2


# 0-layer Deep Q-Network

In [0]:
# Q-value table
Q_1 = [[0,0],[0,0],[0,0],[0,0],[0,0]]
Q_2 = [[0,0],[0,0],[0,0],[0,0],[0,0]]

In [11]:
class RNN():
  
  def __init__(self, input_dim, n_hidden):
    # x = e.g. [[2,2], [1,0], ...]
    # h = LSTM(x) (1 x n_hidden)
    # output = Wh + b (1 x 2)
    
    self.input_dim = input_dim
    
    # Weight initialization
    self.W = tf.Variable(tf.random_normal([n_hidden, input_dim]))
    self.b = tf.Variable(tf.random_normal([input_dim]))
    
    self.rnn_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

  def forward(self, x):
    # generate prediction
    outputs, states = tf.contrib.rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are input_dim outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], W) + b
  
a = np.zeros([3,2])
a[0][0] = 2
a[1][1] = 3
a[2][0] = 5
print(a)

print(np.amax(a,axis=1))

b = np.zeros([1,2])
np.concatenate((a[1:,:],np.zeros([1,2])),axis=0)

c = np.array([1,2])
print(c)
c = [c]
print(np.flip(c,axis=0))
print(c[-1][::-1])

[[2. 0.]
 [0. 3.]
 [5. 0.]]
[2. 3. 5.]
[1 2]
[[1 2]]
[2 1]


# Game

In [31]:
sess = tf.InteractiveSession() # Initialize session

# Q-networks
RNN1 = RNN(input_dim, n_hidden)
RNN2 = RNN(input_dim, n_hidden)

nextQ1 = tf.placeholder(shape=[n_turns,2],dtype=tf.float32)
x1 = tf.placeholder(shape=[1,2],dtype=tf.float32)
Qout1 = RNN1.forward(x)

nextQ2 = tf.placeholder(shape=[n_turns,2],dtype=tf.float32)
x2 = tf.placeholder(shape=[1,2],dtype=tf.float32)
Qout2 = RNN2.forward(x)

# Loss and optimizer
cost1 = tf.reduce_sum(tf.square(nextQ1-Qout1))
optimizer1 = tf.train.RMSPropOptimizer(learning_rate=alpha).minimize(cost1)

cost2 = tf.reduce_sum(tf.square(nextQ2-Qout2))
optimizer2 = tf.train.RMSPropOptimizer(learning_rate=alpha).minimize(cost2)

sess.run(tf.global_variables_initializer()) # Initialize variables

for i_ep in range(n_eps):
  # History of outcomes
  history = [np.array([2,2])] # Start
  
  # Keep track of calculated Q-values
  Q_pred1 = np.zeros([n_turns,2])
  Q_pred2 = np.zeros([n_turns,2])
  
  # Keep track of rewards
  rewards1 = np.zeros([n_turns,1])
  rewards2 = np.zeros([n_turns,1])
  
  for i_turn in range(n_turns):
    
    # Calculate Q-values
    Q1 = sess.run(Qout, feed_dict={x: history, agentID: 1})
    Q2 = sess.run(Qout, feed_dict={x: [outcome[::-1] for outcome in history], agentID: 2})
    
    Q_pred1[i_turn] = Q1
    Q_pred2[i_turn] = Q2
    
    # Epsilon-greedy sampling
    if np.random.rand(1) < epsilon:
      action1 = np.random.randint(2)
    else:
      action1 = np.argmax(Q1)
      
    if np.random.rand(1) < epsilon:
      action2 = np.random.randint(2)
    else:
      action2 = np.argmax(Q2)
      
    # Get rewards
    reward1 = RM[action1][action2][0]
    reward2 = RM[action1][action2][1]
    
    rewards1[i_turn] = reward1
    rewards2[i_turn] = reward2
    
    # Add to history
    history.append([action1,action2])
    
  # Q-learning
  
  # Q_{s+1}
  Q_pred_shift1 = np.concatenate((Q_pred1[1:,:],np.zeros([1,2])),axis=0)
  Q_target1 = rewards1 + gamma * np.amax(Q_pred_shift1,axis=1)
  
  Q_pred_shift2 = np.concatenate((Q_pred2[1:,:],np.zeros([1,2])),axis=0)
  Q_target2 = rewards2 + gamma * np.amax(Q_pred_shift2,axis=1)
  
  sess.run(optimizer1, feed_dict={x1:history, nextQ1:Q_target1})
  sess.run(optimizer2, feed_dict={x2:[outcome[::-1] for outcome in history], nextQ2:Q_target2})

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


NameError: ignored

In [5]:
# Previous outcome lookup policy for both players
p_1 = [0.3, 0.4, 0.5, 0.2]
p_2 = [0.4, 0.3, 0.2, 0.5] # Daniel's pick

# Reward for both players
R1 = 0
R2 = 0

outcome = 4 # 0 = CC, 1 = CD, 2 = DC, 3 = DD, 4 = Start
for n in range(N):
  
  # Boltzmann sampling using Q-values
  # Probability of collaborating
  p_1 = np.exp(Q_1[outcome][0])/np.sum(np.exp(Q_1[outcome]))
  p_2 = np.exp(Q_2[outcome][0])/np.sum(np.exp(Q_2[outcome]))
  
  r1 = np.random.rand()
  r2 = np.random.rand()
  
  if p_1 > r1 and p_2 > r2:
    
    next_outcome = 0
    action1 = 0
    action2 = 0
    reward1 = R
    reward2 = R
    
  elif p_1 > r1 and p_2 <= r2:
    
    next_outcome = 1
    action1 = 0
    action2 = P
    reward1 = 0
    reward2 = T
    
  elif p_1 <= r1 and p_2 > r2:
    
    next_outcome = 2
    action1 = 1
    action2 = 0
    reward1 = T
    reward2 = S
    
  elif p_1 <= r1 and p_2 <= r2:
    
    next_outcome = 3
    action1 = 1
    action2 = 1
    reward1 = P
    reward2 = P
    
  R1 += reward1
  R2 += reward2
    
  # Q-learning <--- HERE
  Q_1[outcome][action1] += alpha * (reward1 + gamma*np.max(Q_1[next_outcome]) - Q_1[outcome][action1])
  Q_2[outcome][action2] += alpha * (reward2 + gamma*np.max(Q_2[next_outcome]) - Q_2[outcome][action2])
  
  outcome = next_outcome
    
print(R1)
print(R2)

1021
1126


In [6]:
Q_1

[[0.9433724908263161, 0.5659474045908255],
 [1.0956443850857551, 2.757813922629177],
 [0.5001720490563554, 0.6315677743814965],
 [1.7888472409760967, 9.999341552075956],
 [0, 0.1]]

In [0]:
p_1 = np.exp(Q_1[outcome][0])/np.sum(np.exp(Q_1[outcome]))

In [8]:
p_1

0.00027171249332988586

In [9]:
outcome

3

#Message Network

In [12]:
#Create a message
#First position is what you did last turn: 0 = c, 1 = d
#Next n many characters are a message from each agent
#m1 is agent 1 message and m2 is agent 2 message

n = 3
m1 = np.zeros(n+2)
m2 = np.zeros(n+2)

m1[0] = action1
m2[0] = action2
m1[1] = action2
m2[2] = action1

print(m1)

[1. 1. 0. 0. 0.]
